In [ ]:
from scipy.sparse import issparse

import numpy as np
import pandas as pd
import scanpy as sc

from liana.method.sp._spatial_pipe import spatial_neighbors
from liana.method.sp._misty import _get_neighbors

from scipy.sparse import csr_matrix
from mudata import MuData
from anndata import AnnData
import os

In [ ]:
from liana.method._pipe_utils import prep_check_adata

In [ ]:
from liana.mt.sp._misty import _check_target_in_predictors, _check_features, _single_view_model, \
    _mask_connectivity, _multi_model, _format_targets, _format_importances, _concat_dataframes, _check_if_squidpy

In [ ]:
def _make_view(adata, connecitivity=None, spatial_key=None, layer=None, use_raw=False, obs=None):
    
    adata = prep_check_adata(adata, use_raw=use_raw, layer=layer, groupby=None, min_cells=None)    
    if connecitivity is not None:
        conns = connecitivity
        obsp = {'spatial_connectivities': conns}
    elif spatial_key is not None:
        conns = adata.obsp[spatial_key]
        obsp = {'spatial_connectivities': conns}
    else:
        obsp=None
        
    return AnnData(X=adata.X, obs=obs, var=adata.var, obsp=obsp)

In [ ]:
adata = sc.read_h5ad('liana/tests/data/synthetic.h5ad')

MistyData class

In [ ]:
class MistyData(MuData):

    def __init__(self, views, obs):
        MuData.__init__(self, views)
        self.view_names = list(self.mod.keys())
        self._check_views()
        self.obs = obs
    
    def _check_views(self):
        assert isinstance(self, MuData), "views must be a MuData object"
        assert "intra" in self.view_names, "views must contain an intra view"
        
        for view in self.view_names:
            if view=="intra":
                continue
            if "spatial_connectivities" not in self.mod[view].obsp.keys():
                raise ValueError(f"view {view} must contain a spatial_connectivities key in .obsp")
    
    ## TODO assign targets & predictors to views
    
            
    def _get_conn(self, view_name):
        return self.mod[view_name].obsp["spatial_connectivities"]

 

### Constructor

In [ ]:
# xdata = adata

In [ ]:
ydata = None
set_diag = True
spatial_key = 'spatial'
bandwidth = 10
n_neighs = 6
kernel = 'misty_rbf'
cutoff = 0.1
juxta_cutoff = np.inf
zoi = 0

In [ ]:
# ydata = ydata if ydata else xdata

In [ ]:
def construct_views(xdata, 
                    ydata=None,
                    add_para=True,
                    spatial_key='spatial',
                    set_diag=True, 
                    bandwidth = 10,
                    kernel = 'misty_rbf',
                    zoi = 0,
                    cutoff = 0.1,
                    add_juxta=True,
                    n_neighs = 6,
                    juxta_cutoff = np.inf,
                    **kwargs
                    ):
    
    # init views
    views = {}
    ydata = ydata if ydata else xdata
    
    # NOTE the intra view is the one with obs
    views['intra'] = _make_view(adata=ydata, use_raw=False, layer=None)
    
    if add_para:
        weights = spatial_neighbors(adata=xdata,
                                    bandwidth=bandwidth,
                                    kernel=kernel,
                                    set_diag=set_diag, 
                                    inplace=False,
                                    cutoff=cutoff,
                                    zoi=zoi
                                    )
        views['para'] = _make_view(xdata, use_raw=False, layer=None, connecitivity=weights)

    if add_juxta:
        sq = _check_if_squidpy()
        neighbors, dists = sq.gr.spatial_neighbors(adata=xdata,
                                                   copy=True,
                                                   spatial_key=spatial_key,
                                                   set_diag=set_diag,
                                                   n_neighs=n_neighs,
                                                   **kwargs
                                                   )
        neighbors[dists > juxta_cutoff] = 0

        views['juxta'] = _make_view(xdata, use_raw=False, layer=None, connecitivity=neighbors)
    
    return views, ydata.obs



In [ ]:
views, obs = construct_views(adata, delaunay=True, coord_type="generic")

In [ ]:
mdata = MistyData(views, obs)

In [ ]:
mdata

In [ ]:
mdata.view_names

In [ ]:
## TODO: two applications
# juxta, para
# misty_lr
# anything else would require a new constructor

In [ ]:
predictors = None
targets = None

In [ ]:
intra = mdata.mod['intra']

In [ ]:
# TODO to be abstracted further
predictors = _check_features(intra, predictors, type_str="predictors")
targets = _check_features(intra, targets, type_str="targets")

In [ ]:
targets

#### FIT params

In [ ]:
n_estimators = 100
n_jobs = -1
seed = 1337
bypass_intra = True
keep_same_predictor = False
k_cv = 10
alphas = [0.1, 1, 10]

In [ ]:
group_intra_by = None
group_env_by = None

In [ ]:
# TODO: function that checks if the groupby is in the obs
# and does this for both extra & intra
intra_groups = np.unique(mdata.obs[group_intra_by]) if group_intra_by else [None]
extra_groups = np.unique(mdata.obs[group_env_by]) if group_env_by else [None]

In [ ]:
view_str = list(mdata.view_names)
if bypass_intra:
    view_str.remove('intra')

In [ ]:
# init list to store the results for each intra group and env group as dataframe;
targets_list, importances_list = [], []

In [ ]:
# loop over each target and build one RF model for each view
for target in targets:
    
    for intra_group in intra_groups:
        intra_obs_msk = intra.obs[group_intra_by] == \
                intra_group if intra_group else np.ones(intra.shape[0], dtype=bool)
        
        # to array
        y = intra[intra_obs_msk, target].X.toarray().reshape(-1)
        
        # intra is always non-self, while other views can be self
        predictors_nonself, insert_index = _check_target_in_predictors(target, predictors)
        _predictors = predictors if keep_same_predictor else predictors_nonself

        # TODO: rename to target_importances
        importance_dict = {}
        
        # model the intraview
        if not bypass_intra:
            oob_predictions_intra, importance_dict["intra"] = _single_view_model(y,
                                                                                 intra,
                                                                                 intra_obs_msk, 
                                                                                 predictors_nonself, 
                                                                                 n_estimators, 
                                                                                 n_jobs, 
                                                                                 seed
                                                                                 )
            if insert_index is not None and keep_same_predictor:
                importance_dict["intra"] = np.insert(importance_dict["intra"], insert_index, np.nan)

        # loop over the group_views_by
        for extra_group in extra_groups:
            # store the oob predictions for each view to construct predictor matrix for meta model
            oob_list = []

            if not bypass_intra:
                oob_list.append(oob_predictions_intra)

            # model the juxta and paraview (if applicable)
            ## TODO: remove this thing with all
            for view_name in [v for v in view_str if v != "intra"]:
                extra = mdata.mod[view_name]
                # TODO rename to extra_obs_msk
                env_obs_msk = mdata.obs[group_env_by] == extra_group if extra_group else np.ones(extra.shape[0], dtype=bool)
                
                # NOTE indexing here is expensive, but we do it to avoid memory issues
                connectivity = mdata._get_conn(view_name)
                view = _mask_connectivity(extra, connectivity, env_obs_msk, _predictors)
                
                oob_predictions, importance_dict[view_name] = \
                    _single_view_model(y,
                                       view, 
                                       intra_obs_msk, 
                                       _predictors, 
                                       n_estimators,
                                       n_jobs,
                                       seed
                                       )
                oob_list.append(oob_predictions)


            # train the meta model with k-fold CV 
            intra_r2, multi_r2, coefs = _multi_model(y,
                                                    np.column_stack(oob_list),
                                                    intra_group, 
                                                    bypass_intra, 
                                                    view_str, 
                                                    k_cv, 
                                                    alphas, 
                                                    seed
                                                    )
            
            targets_df = _format_targets(target,
                                        intra_group,
                                        extra_group,
                                        view_str,
                                        intra_r2,
                                        multi_r2,
                                        coefs
                                        )
            targets_list.append(targets_df)
            
            importances_df = _format_importances(target, 
                                                _predictors, 
                                                intra_group, 
                                                extra_group,
                                                importance_dict
                                                )
            importances_list.append(importances_df)


# create result dataframes
target_metrics, importances = _concat_dataframes(targets_list,
                                                 importances_list,
                                                 view_str)

In [ ]:
target_metrics

In [ ]:
importances

In [ ]:
views